In [6]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from uvicorn import run
app = FastAPI()

import asyncio
import uvicorn


from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from trt_inference_api import CustomLLM 

In [2]:
token_path = "/trt-llm/mistral-7b-int4-chat_1.2/mistral7b_hf_tokenizer"
engine_path = "/trt-llm/mistral-7b-int4-chat_1.2/trt_engines"
llm = CustomLLM(n=5,tokenizer_dir = token_path, engine_dir = engine_path)

/home/adebolajo/Desktop/trt-llm/mistral-7b-int4-chat_1.2/trt_engines
/home/adebolajo/Desktop/trt-llm/mistral-7b-int4-chat_1.2/mistral7b_hf_tokenizer


In [3]:
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/home/adebolajo/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
class Message(BaseModel):
    content: str

In [8]:
app = FastAPI()
# Allow CORS for all origins, methods, and headers
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "OPTIONS"],  # Allow OPTIONS method
    allow_headers=["*"],
)

# Mount the static directory to serve static files
app.mount("/static", StaticFiles(directory="/backend/static"), name="static")

# Create a Jinja2Templates instance for rendering HTML templates
templates = Jinja2Templates(directory="/backend/templates")

@app.get("/",response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("fred.html", {"request": request})


@app.post("/chat")
async def chat_completion(message: Message):
    # Your logic to generate a response based on the received message
    response = "This is a sample response."
    msg = message.content
    response = conversation({"question": msg})
    print( msg, response['text'])


    #return {"response": "Sure, I can help you with that. Here's a simple FastAPI code that you can use as a starting point:\n```python\nfrom fastapi import FastAPI\n\napp = FastAPI()\n\n@app.get(\"/\")\nasync def read_root():\n    return {\"Hello\": \"World\"}\n```\nThis code creates a FastAPI application and defines a route for the root endpoint. When you run this code, you can access the root endpoint by visiting `http://localhost:8000/` in your web browser.\nIs there anything else you need help with?"}
    return {"response": response['text']}



if __name__ == "__main__":
    config = uvicorn.Config(app,host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()


INFO:     Started server process [16195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50016 - "GET / HTTP/1.1" 200 OK


> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi 
AI:  Hello! How can I help you today?

Human: what happened to the world in 2020
AI:  

In 2020, the world experienced a pandemic caused by the novel coronavirus (COVID-19). The virus spread rapidly and caused widespread illness and death. Many countries implemented lockdowns and other measures to try to slow the spread of the virus. The pandemic also had a significant impact on the global economy, causing widespread job losses and business closures. Additionally, there were protests and unrest in many parts of the world, including the United States, where there were widespread protests against the killing of George Floyd.


New human question: hi
Response:
Input [Text 0]: "<s> You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi 
AI:  Hello! H

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16195]
